# Spider Workshop - Exercise 5: Constrained Inverse Kinematics
(These exercises were prepared for ganja.js by Steven De Keninck for [GAME23](https://bivector.net/game2023.html).)

We will learn : 
1. How to constrain our leg to a plane, so it looks more spider-like in 3D
2. How to implement a distance constraint
3. How to implement an angle constraint

In [1]:
%pip install -q kingdon anywidget==0.9.9 ipywidgets==8.1.3

Note: you may need to restart the kernel to use updated packages.


Create a geometric algebra with 3 positive and one null basis vector:

In [2]:
from kingdon import Algebra

alg = Algebra(3, 0, 1)

In lesson 4 we learned how to translate along a line. Let's make this into a function:

In [3]:
def tr(line, dist: float):
    """ Translate a distance along a line. """
    horizon = alg.blades.e0
    origin = horizon.dual()
    return ( (-0.5 * dist * horizon) ^ (line.normalized() | origin)).exp()

Our chain will have three segments, so four points, 0.5 apart. Let's also define a base and a target point.

In [4]:
chain = [
    alg.vector(e0=1, e1=0, e2=0, e3=0).dual(),
    alg.vector(e0=1, e1=0.5, e2=0, e3=0).dual(),
    alg.vector(e0=1, e1=1, e2=0, e3=0).dual(),
    alg.vector(e0=1, e1=1.5, e2=0, e3=0).dual(),
]
base = alg.vector(e0=1, e1=-0.5, e2=0, e3=0).dual()
target = alg.vector(e0=1, e1=1, e2=-1, e3=0).dual()

Our job will be to complete the IK function below.

In [5]:
def inverse_kinematics(C, Base, Target) -> None:
    """Our IK function takes a chain, a base and a target, and updates the chain *in place* to a new chain."""
    C[3] = Target
    for i in reversed(range(3)): 
        C[i] = tr(C[i] & C[i+1], 0.5) >> C[i+1]
    
    # Here we put our constraints
    
    C[0] = Base
    for i in range(1, 3 + 1):
        C[i] = tr(C[i-1] & C[i], -0.5) >> C[i-1]

Render these elements using the `Algebra.graph` function. If you provide a function without arguments to `Algebra.graph`, the function will be re-evaluated everytime you drag a point.
So pro-tip for the coming exercises: define everything within `graph_func`. 

In [6]:
def graph_func():
    inverse_kinematics(chain, base, target)
    
    return [
        "Spider Workshop - Constrained Inverse Kinematics", 
        [chain[0], chain[1]],
        [chain[1], chain[2]],
        [chain[2], chain[3]],
        0xFF0000, base, "B", target, "T",
    ]

alg.graph(
    graph_func,
    grid=True, labels=True, lineWidth=4,
)

GraphWidget(cayley=[['1', 'e0', 'e1', 'e2', 'e3', 'e01', 'e02', 'e03', 'e12', 'e13', 'e23', 'e012', 'e013', 'e…

## Exercises:

First test the existing solution in 3D, especially after turning the camera. Note how the points
of the leg do not stay in the same plane, giving a non-spider look. Let's fix that by constraining
these points to lie in a vertical plane.

1. Below line 7 of the `inverse_kinematics` function, construct `plane` containing Base and Target by using two joins (&) containing the Base, Target and the point at infinity in the "up" direction (e2.dual()).

2. Project all points in a for loop to this plane using the projection formula `(x|plane)/plane`.
   Examine how this changes the behavior as you drag the base and target.
   What would we break if we did this projection as the last step?

3. We also want to make sure the first joint point never goes below the base point. To do this
   create a `plane2`, through the base point, orthogonal to the up-line (e13).

4. Measure the signed distance from the first joint (`C[1]`) to this plane, by joining `C[1]` and
   `plane2` to make a volume. If this volume is negative, the first joint is under the base
   point and you want to project this joint point `C[1]` back onto `plane2`.
    (hint: to compare the signed distance you should extract its scalar component using '.e': 
      distance = (point & plane).e )


## Extra Credit

5. Make sure the last joint can only bend one way by measuring the angle between the two last
   segments. If it is negative, mirror `C[2]` to the other side of the line between `C[1]` and `C[3]`.
   Hint : the angle you want is `(((C[1]&C[2]|C[2]) ^ plane) | (C[2]&C[3])).e`.
   This construction creates a line orthogonal to the middle segment `((C[1]&C[2]|C[2]) ^ plane)` and 
   dots that with the line of the last segment, returning the cosine of the angle. 


In [7]:
from exercises.solution_widget import SolutionWidget

SolutionWidget(exercise='spider5')

SolutionWidget()